In [0]:
from databricksx12 import *
from databricksx12.hls import *
import json, os
from pyspark.sql.functions import input_file_name


In [0]:
# Initialize HealthcareManager
hm = HealthcareManager()
df = spark.read.text("/Volumes/raven_catalog/x12_edi_parser/sampledata/837/*.txt", wholetext = True)

# Process the DataFrame and convert to RDD
rdd = (
 df.withColumn("filename", input_file_name()).rdd
  .map(lambda x: (x.asDict().get("filename"),x.asDict().get("value")))
  .map(lambda x: (x[0], EDI(x[1])))
  .map(lambda x: { **{'filename': x[0]}, **hm.to_json(x[1])} )
  .map(lambda x: json.dumps(x))
)

edi_data = spark.read.json(rdd)

In [0]:
display(edi_data)

EDI.sender_tax_id FuncitonalGroup filename ZZ List(List(123456789, CLEARINGHOUSE, 20180508:0833, 222, List(List(List(List(List(, , 20, 1805080AV3648339, , , 57:B:1, , ), List(List(1, 1, 20, , , H0003, HC, null, 20180428, D8, 472, 1, UN)), List(, , , F1120, ), List(VANCOUVER, 19881225, D8, M, SUBSCRIBER JOHN J, , WA, 987 65TH PL, 18, 986640001), List(2, CHPWA, PI, COMMUNITY HEALTH PLAN OF WASHINGTON), List(null, List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 12345 MAIN ST, , 98662), null, null, null, List(, , , Individual, , , , , , , ), List(, , , Organization, BH CLINIC OF VANCOUVER, 1122334455, , , , , ), List(, , , Individual, PROVIDER JAMES , 1112223338, PE, , , 261QR0405X, )), List(123456789, Organization), List(CLEARINGHOUSE LLC, CLEARINGHOUSE CLIENT SERVICES, List(List(TE, 8005551212), List(FX, 8005551212)), Organization), List(VANCOUVER, 19881225, D8, M, SUBSCRIBER JOHN J, , WA, 987 65TH PL, 18, 986640001))), 222), List(List(List(List(, , 50.1, 1805080AV3648347, , , 57:B:1, , ), List(List(1, 1:2, 50.1, HF, , 96153, HC, null, 20180426, D8, 472, 6, UN)), List(, , , F1520, ), List(VANCOUVER, 19881031, D8, F, PATIENT SUSAN E, , WA, 765 43RD ST, 18, 986640002), List(2, CHPWA, PI, COMMUNITY HEALTH PLAN OF WASHINGTON), List(null, List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 12345 MAIN ST, , 98662), null, null, null, List(, , , Individual, , , , , , , ), List(, , , Organization, BH CLINIC OF VANCOUVER, 1122334455, , , , , ), List(, , , Individual, PROVIDER SUSAN , 1112223346, PE, , , 261QR0405X, )), List(123456789, Organization), List(CLEARINGHOUSE LLC, CLEARINGHOUSE CLIENT SERVICES, List(List(TE, 8005551212), List(FX, 8005551212)), Organization), List(VANCOUVER, 19881031, D8, F, PATIENT SUSAN E, , WA, 765 43RD ST, 18, 986640002))), 222), List(List(List(List(, , 11.64, 1805080AV3648340, , , 57:B:1, , ), List(List(1, 1:2, 11.64, HF, , T1017, HC, null, 20180427, D8, 472, 1, UN)), List(, , , F1020, ), List(VANCOUVER, 19881225, D8, M, SUBSCRIBER JOHN J, , WA, 987 65TH PL, 18, 986640001), List(2, CHPWA, PI, COMMUNITY HEALTH PLAN OF WASHINGTON), List(null, List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 12345 MAIN ST, , 98662), null, null, null, List(, , , Individual, , , , , , , ), List(, , , Organization, BH CLINIC OF VANCOUVER, 1122334455, , , , , ), List(, , , Individual, PROVIDER SUSAN , 1112223346, PE, , , 261QR0405X, )), List(123456789, Organization), List(CLEARINGHOUSE LLC, CLEARINGHOUSE CLIENT SERVICES, List(List(TE, 8005551212), List(FX, 8005551212)), Organization), List(VANCOUVER, 19881225, D8, M, SUBSCRIBER JOHN J, , WA, 987 65TH PL, 18, 986640001))), 222), List(List(List(List(, , 234, 1805080AV3648353, , , 53:B:1, , ), List(List(1, 1, 234, , , 90853, HC, null, 20180427, D8, 472, 120, UN)), List(, , , F251, ), List(VANCOUVER, 19881031, D8, F, PATIENT SUSAN E, , WA, 765 43RD ST, 18, 986640002), List(2, CHPWA, PI, COMMUNITY HEALTH PLAN OF WASHINGTON), List(null, List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 12345 MAIN ST, , 98662), null, null, null, List(, , , Individual, , , , , , , ), List(, , , Organization, BH CLINIC OF VANCOUVER, 1122334455, , , , , ), List(, , , Individual, PROVIDER SUSAN , 1112223346, PE, , , 251S00000X, )), List(123456789, Organization), List(CLEARINGHOUSE LLC, CLEARINGHOUSE CLIENT SERVICES, List(List(TE, 8005551212), List(FX, 8005551212)), Organization), List(VANCOUVER, 19881031, D8, F, PATIENT SUSAN E, , WA, 765 43RD ST, 18, 986640002))), 222), List(List(List(List(, , 20, 1805080AV3648355, , , 57:B:1, , ), List(List(1, 1:2, 20, , , H0003, HC, null, 20180427, D8, 472, 1, UN)), List(, , , F1020, ), List(VANCOUVER, 19881225, D8, M, SUBSCRIBER JOHN J, , WA, 987 65TH PL, 18, 986640001), List(2, CHPWA, PI, COMMUNITY HEALTH PLAN OF WASHINGTON), List(null, List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 123

In [0]:
from pyspark.sql.functions import explode, col
from pyspark.sql.types import StructType, ArrayType

# Extract FunctionalGroups and Transactions
functional_groups_df = edi_data.select(
  col("`EDI.sender_tax_id`").alias("tax_id"),
  explode(col("FuncitonalGroup")).alias("FunctionalGroup"))

display(functional_groups_df)

tax_id FunctionalGroup ZZ List(123456789, CLEARINGHOUSE, 20180508:0833, 222, List(List(List(List(List(, , 20, 1805080AV3648339, , , 57:B:1, , ), List(List(1, 1, 20, , , H0003, HC, null, 20180428, D8, 472, 1, UN)), List(, , , F1120, ), List(VANCOUVER, 19881225, D8, M, SUBSCRIBER JOHN J, , WA, 987 65TH PL, 18, 986640001), List(2, CHPWA, PI, COMMUNITY HEALTH PLAN OF WASHINGTON), List(null, List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 12345 MAIN ST, , 98662), null, null, null, List(, , , Individual, , , , , , , ), List(, , , Organization, BH CLINIC OF VANCOUVER, 1122334455, , , , , ), List(, , , Individual, PROVIDER JAMES , 1112223338, PE, , , 261QR0405X, )), List(123456789, Organization), List(CLEARINGHOUSE LLC, CLEARINGHOUSE CLIENT SERVICES, List(List(TE, 8005551212), List(FX, 8005551212)), Organization), List(VANCOUVER, 19881225, D8, M, SUBSCRIBER JOHN J, , WA, 987 65TH PL, 18, 986640001))), 222), List(List(List(List(, , 50.1, 1805080AV3648347, , , 57:B:1, , ), List(List(1, 1:2, 50.1, HF, , 96153, HC, null, 20180426, D8, 472, 6, UN)), List(, , , F1520, ), List(VANCOUVER, 19881031, D8, F, PATIENT SUSAN E, , WA, 765 43RD ST, 18, 986640002), List(2, CHPWA, PI, COMMUNITY HEALTH PLAN OF WASHINGTON), List(null, List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 12345 MAIN ST, , 98662), null, null, null, List(, , , Individual, , , , , , , ), List(, , , Organization, BH CLINIC OF VANCOUVER, 1122334455, , , , , ), List(, , , Individual, PROVIDER SUSAN , 1112223346, PE, , , 261QR0405X, )), List(123456789, Organization), List(CLEARINGHOUSE LLC, CLEARINGHOUSE CLIENT SERVICES, List(List(TE, 8005551212), List(FX, 8005551212)), Organization), List(VANCOUVER, 19881031, D8, F, PATIENT SUSAN E, , WA, 765 43RD ST, 18, 986640002))), 222), List(List(List(List(, , 11.64, 1805080AV3648340, , , 57:B:1, , ), List(List(1, 1:2, 11.64, HF, , T1017, HC, null, 20180427, D8, 472, 1, UN)), List(, , , F1020, ), List(VANCOUVER, 19881225, D8, M, SUBSCRIBER JOHN J, , WA, 987 65TH PL, 18, 986640001), List(2, CHPWA, PI, COMMUNITY HEALTH PLAN OF WASHINGTON), List(null, List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 12345 MAIN ST, , 98662), null, null, null, List(, , , Individual, , , , , , , ), List(, , , Organization, BH CLINIC OF VANCOUVER, 1122334455, , , , , ), List(, , , Individual, PROVIDER SUSAN , 1112223346, PE, , , 261QR0405X, )), List(123456789, Organization), List(CLEARINGHOUSE LLC, CLEARINGHOUSE CLIENT SERVICES, List(List(TE, 8005551212), List(FX, 8005551212)), Organization), List(VANCOUVER, 19881225, D8, M, SUBSCRIBER JOHN J, , WA, 987 65TH PL, 18, 986640001))), 222), List(List(List(List(, , 234, 1805080AV3648353, , , 53:B:1, , ), List(List(1, 1, 234, , , 90853, HC, null, 20180427, D8, 472, 120, UN)), List(, , , F251, ), List(VANCOUVER, 19881031, D8, F, PATIENT SUSAN E, , WA, 765 43RD ST, 18, 986640002), List(2, CHPWA, PI, COMMUNITY HEALTH PLAN OF WASHINGTON), List(null, List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 12345 MAIN ST, , 98662), null, null, null, List(, , , Individual, , , , , , , ), List(, , , Organization, BH CLINIC OF VANCOUVER, 1122334455, , , , , ), List(, , , Individual, PROVIDER SUSAN , 1112223346, PE, , , 251S00000X, )), List(123456789, Organization), List(CLEARINGHOUSE LLC, CLEARINGHOUSE CLIENT SERVICES, List(List(TE, 8005551212), List(FX, 8005551212)), Organization), List(VANCOUVER, 19881031, D8, F, PATIENT SUSAN E, , WA, 765 43RD ST, 18, 986640002))), 222), List(List(List(List(, , 20, 1805080AV3648355, , , 57:B:1, , ), List(List(1, 1:2, 20, , , H0003, HC, null, 20180427, D8, 472, 1, UN)), List(, , , F1020, ), List(VANCOUVER, 19881225, D8, M, SUBSCRIBER JOHN J, , WA, 987 65TH PL, 18, 986640001), List(2, CHPWA, PI, COMMUNITY HEALTH PLAN OF WASHINGTON), List(null, List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 12345 MAIN ST, , 98662), nul

In [0]:
functional_groups_flat = functional_groups_df.select(
    col("tax_id"),
    col("FunctionalGroup.`FunctionalGroup.receiver`").alias("receiver"),
    col("FunctionalGroup.`FunctionalGroup.sender`").alias("sender"),
    col("FunctionalGroup.`FunctionalGroup.transaction_datetime`").alias("transaction_datetime"),
    col("FunctionalGroup.`FunctionalGroup.transaction_type`").alias("transaction_type"),
    explode(col("FunctionalGroup.Transactions")).alias("Transaction")
)

display(functional_groups_flat)

tax_id receiver sender transaction_datetime transaction_type Transaction ZZ 123456789 CLEARINGHOUSE 20180508:0833 222 List(List(List(List(, , 20, 1805080AV3648339, , , 57:B:1, , ), List(List(1, 1, 20, , , H0003, HC, null, 20180428, D8, 472, 1, UN)), List(, , , F1120, ), List(VANCOUVER, 19881225, D8, M, SUBSCRIBER JOHN J, , WA, 987 65TH PL, 18, 986640001), List(2, CHPWA, PI, COMMUNITY HEALTH PLAN OF WASHINGTON), List(null, List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 12345 MAIN ST, , 98662), null, null, null, List(, , , Individual, , , , , , , ), List(, , , Organization, BH CLINIC OF VANCOUVER, 1122334455, , , , , ), List(, , , Individual, PROVIDER JAMES , 1112223338, PE, , , 261QR0405X, )), List(123456789, Organization), List(CLEARINGHOUSE LLC, CLEARINGHOUSE CLIENT SERVICES, List(List(TE, 8005551212), List(FX, 8005551212)), Organization), List(VANCOUVER, 19881225, D8, M, SUBSCRIBER JOHN J, , WA, 987 65TH PL, 18, 986640001))), 222) ZZ 123456789 CLEARINGHOUSE 20180508:0833 222 List(List(List(List(, , 50.1, 1805080AV3648347, , , 57:B:1, , ), List(List(1, 1:2, 50.1, HF, , 96153, HC, null, 20180426, D8, 472, 6, UN)), List(, , , F1520, ), List(VANCOUVER, 19881031, D8, F, PATIENT SUSAN E, , WA, 765 43RD ST, 18, 986640002), List(2, CHPWA, PI, COMMUNITY HEALTH PLAN OF WASHINGTON), List(null, List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 12345 MAIN ST, , 98662), null, null, null, List(, , , Individual, , , , , , , ), List(, , , Organization, BH CLINIC OF VANCOUVER, 1122334455, , , , , ), List(, , , Individual, PROVIDER SUSAN , 1112223346, PE, , , 261QR0405X, )), List(123456789, Organization), List(CLEARINGHOUSE LLC, CLEARINGHOUSE CLIENT SERVICES, List(List(TE, 8005551212), List(FX, 8005551212)), Organization), List(VANCOUVER, 19881031, D8, F, PATIENT SUSAN E, , WA, 765 43RD ST, 18, 986640002))), 222) ZZ 123456789 CLEARINGHOUSE 20180508:0833 222 List(List(List(List(, , 11.64, 1805080AV3648340, , , 57:B:1, , ), List(List(1, 1:2, 11.64, HF, , T1017, HC, null, 20180427, D8, 472, 1, UN)), List(, , , F1020, ), List(VANCOUVER, 19881225, D8, M, SUBSCRIBER JOHN J, , WA, 987 65TH PL, 18, 986640001), List(2, CHPWA, PI, COMMUNITY HEALTH PLAN OF WASHINGTON), List(null, List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 12345 MAIN ST, , 98662), null, null, null, List(, , , Individual, , , , , , , ), List(, , , Organization, BH CLINIC OF VANCOUVER, 1122334455, , , , , ), List(, , , Individual, PROVIDER SUSAN , 1112223346, PE, , , 261QR0405X, )), List(123456789, Organization), List(CLEARINGHOUSE LLC, CLEARINGHOUSE CLIENT SERVICES, List(List(TE, 8005551212), List(FX, 8005551212)), Organization), List(VANCOUVER, 19881225, D8, M, SUBSCRIBER JOHN J, , WA, 987 65TH PL, 18, 986640001))), 222) ZZ 123456789 CLEARINGHOUSE 20180508:0833 222 List(List(List(List(, , 234, 1805080AV3648353, , , 53:B:1, , ), List(List(1, 1, 234, , , 90853, HC, null, 20180427, D8, 472, 120, UN)), List(, , , F251, ), List(VANCOUVER, 19881031, D8, F, PATIENT SUSAN E, , WA, 765 43RD ST, 18, 986640002), List(2, CHPWA, PI, COMMUNITY HEALTH PLAN OF WASHINGTON), List(null, List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 12345 MAIN ST, , 98662), null, null, null, List(, , , Individual, , , , , , , ), List(, , , Organization, BH CLINIC OF VANCOUVER, 1122334455, , , , , ), List(, , , Individual, PROVIDER SUSAN , 1112223346, PE, , , 251S00000X, )), List(123456789, Organization), List(CLEARINGHOUSE LLC, CLEARINGHOUSE CLIENT SERVICES, List(List(TE, 8005551212), List(FX, 8005551212)), Organization), List(VANCOUVER, 19881031, D8, F, PATIENT SUSAN E, , WA, 765 43RD ST, 18, 986640002))), 222) ZZ 123456789 CLEARINGHOUSE 20180508:0833 222 List(List(List(List(, , 20, 1805080AV3648355, , , 57:B:1, , ), List(List(1, 1:2, 20, , , H0003, HC, null, 20180427, D8, 472, 1, UN)), List(, , , F1020, ), List(VANCOUVER, 19881225, D8, M, SUBSCRIBER JOHN J, , WA, 

In [0]:
claims_flat = functional_groups_flat.select(
    col("tax_id"),
    col("receiver"),
    col("sender"),
    col("transaction_datetime"),
    col("transaction_type"),
    explode(col("Transaction.Claims")).alias("Claim")
)

display(claims_flat)

tax_id,receiver,sender,transaction_datetime,transaction_type,Claim
ZZ,123456789,CLEARINGHOUSE,20180508:0833,222,"List(List(, , 20, 1805080AV3648339, , , 57:B:1, , ), List(List(1, 1, 20, , , H0003, HC, null, 20180428, D8, 472, 1, UN)), List(, , , F1120, ), List(VANCOUVER, 19881225, D8, M, SUBSCRIBER JOHN J, , WA, 987 65TH PL, 18, 986640001), List(2, CHPWA, PI, COMMUNITY HEALTH PLAN OF WASHINGTON), List(null, List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 12345 MAIN ST, , 98662), null, null, null, List(, , , Individual, , , , , , , ), List(, , , Organization, BH CLINIC OF VANCOUVER, 1122334455, , , , , ), List(, , , Individual, PROVIDER JAMES , 1112223338, PE, , , 261QR0405X, )), List(123456789, Organization), List(CLEARINGHOUSE LLC, CLEARINGHOUSE CLIENT SERVICES, List(List(TE, 8005551212), List(FX, 8005551212)), Organization), List(VANCOUVER, 19881225, D8, M, SUBSCRIBER JOHN J, , WA, 987 65TH PL, 18, 986640001))"
ZZ,123456789,CLEARINGHOUSE,20180508:0833,222,"List(List(, , 50.1, 1805080AV3648347, , , 57:B:1, , ), List(List(1, 1:2, 50.1, HF, , 96153, HC, null, 20180426, D8, 472, 6, UN)), List(, , , F1520, ), List(VANCOUVER, 19881031, D8, F, PATIENT SUSAN E, , WA, 765 43RD ST, 18, 986640002), List(2, CHPWA, PI, COMMUNITY HEALTH PLAN OF WASHINGTON), List(null, List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 12345 MAIN ST, , 98662), null, null, null, List(, , , Individual, , , , , , , ), List(, , , Organization, BH CLINIC OF VANCOUVER, 1122334455, , , , , ), List(, , , Individual, PROVIDER SUSAN , 1112223346, PE, , , 261QR0405X, )), List(123456789, Organization), List(CLEARINGHOUSE LLC, CLEARINGHOUSE CLIENT SERVICES, List(List(TE, 8005551212), List(FX, 8005551212)), Organization), List(VANCOUVER, 19881031, D8, F, PATIENT SUSAN E, , WA, 765 43RD ST, 18, 986640002))"
ZZ,123456789,CLEARINGHOUSE,20180508:0833,222,"List(List(, , 11.64, 1805080AV3648340, , , 57:B:1, , ), List(List(1, 1:2, 11.64, HF, , T1017, HC, null, 20180427, D8, 472, 1, UN)), List(, , , F1020, ), List(VANCOUVER, 19881225, D8, M, SUBSCRIBER JOHN J, , WA, 987 65TH PL, 18, 986640001), List(2, CHPWA, PI, COMMUNITY HEALTH PLAN OF WASHINGTON), List(null, List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 12345 MAIN ST, , 98662), null, null, null, List(, , , Individual, , , , , , , ), List(, , , Organization, BH CLINIC OF VANCOUVER, 1122334455, , , , , ), List(, , , Individual, PROVIDER SUSAN , 1112223346, PE, , , 261QR0405X, )), List(123456789, Organization), List(CLEARINGHOUSE LLC, CLEARINGHOUSE CLIENT SERVICES, List(List(TE, 8005551212), List(FX, 8005551212)), Organization), List(VANCOUVER, 19881225, D8, M, SUBSCRIBER JOHN J, , WA, 987 65TH PL, 18, 986640001))"
ZZ,123456789,CLEARINGHOUSE,20180508:0833,222,"List(List(, , 234, 1805080AV3648353, , , 53:B:1, , ), List(List(1, 1, 234, , , 90853, HC, null, 20180427, D8, 472, 120, UN)), List(, , , F251, ), List(VANCOUVER, 19881031, D8, F, PATIENT SUSAN E, , WA, 765 43RD ST, 18, 986640002), List(2, CHPWA, PI, COMMUNITY HEALTH PLAN OF WASHINGTON), List(null, List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 12345 MAIN ST, , 98662), null, null, null, List(, , , Individual, , , , , , , ), List(, , , Organization, BH CLINIC OF VANCOUVER, 1122334455, , , , , ), List(, , , Individual, PROVIDER SUSAN , 1112223346, PE, , , 251S00000X, )), List(123456789, Organization), List(CLEARINGHOUSE LLC, CLEARINGHOUSE CLIENT SERVICES, List(List(TE, 8005551212), List(FX, 8005551212)), Organization), List(VANCOUVER, 19881031, D8, F, PATIENT SUSAN E, , WA, 765 43RD ST, 18, 986640002))"
ZZ,123456789,CLEARINGHOUSE,20180508:0833,222,"List(List(, , 20, 1805080AV3648355, , , 57:B:1, , ), List(List(1, 1:2, 20, , , H0003, HC, null, 20180427, D8, 472, 1, UN)), List(, , , F1020, ), List(VANCOUVER, 19881225, D8, M, SUBSCRIBER JOHN J, , WA, 987 65TH PL, 18, 986640001), List(2, CHPWA, PI, COMMUNITY HEALTH PLAN OF WA

In [0]:
claim_header_flat = claims_flat.selectExpr(
    "tax_id",
    "receiver",
    "sender",
    "transaction_datetime",
    "transaction_type",
    "Claim.claim_header.*",
    "Claim.diagnosis.*",
    "Claim.patient.*",
    "Claim.payer.*",
    "Claim.providers.*"
)

display(claim_header_flat)

tax_id,receiver,sender,transaction_datetime,transaction_type,admission_src_cd,admission_type,claim_amount,claim_id,date_format,discharge_status_cd,facility_type_code,service_date,service_time,admitting_dx_cd,external_injury_dx_cd,other_dx_cds,principal_dx_cd,reason_visit_dx_cd,city,dob,dob_format,gender_cd,name,patient_relationship_cd,state,street,subscriber_relationship_cd,zip,business_entity_type,payer_identifier,payer_identifier_cd,payer_name,attending,billing,facility,operating,other,referring,service_facility,servicing
ZZ,123456789,CLEARINGHOUSE,20180508:0833,222,,,20,1805080AV3648339,,,57:B:1,,,,,,F1120,,VANCOUVER,19881225,D8,M,SUBSCRIBER JOHN J,,WA,987 65TH PL,18,986640001,2,CHPWA,PI,COMMUNITY HEALTH PLAN OF WASHINGTON,null,"List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 12345 MAIN ST, , 98662)",null,null,null,"List(, , , Individual, , , , , , , )","List(, , , Organization, BH CLINIC OF VANCOUVER, 1122334455, , , , , )","List(, , , Individual, PROVIDER JAMES , 1112223338, PE, , , 261QR0405X, )"
ZZ,123456789,CLEARINGHOUSE,20180508:0833,222,,,50.1,1805080AV3648347,,,57:B:1,,,,,,F1520,,VANCOUVER,19881031,D8,F,PATIENT SUSAN E,,WA,765 43RD ST,18,986640002,2,CHPWA,PI,COMMUNITY HEALTH PLAN OF WASHINGTON,null,"List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 12345 MAIN ST, , 98662)",null,null,null,"List(, , , Individual, , , , , , , )","List(, , , Organization, BH CLINIC OF VANCOUVER, 1122334455, , , , , )","List(, , , Individual, PROVIDER SUSAN , 1112223346, PE, , , 261QR0405X, )"
ZZ,123456789,CLEARINGHOUSE,20180508:0833,222,,,11.64,1805080AV3648340,,,57:B:1,,,,,,F1020,,VANCOUVER,19881225,D8,M,SUBSCRIBER JOHN J,,WA,987 65TH PL,18,986640001,2,CHPWA,PI,COMMUNITY HEALTH PLAN OF WASHINGTON,null,"List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 12345 MAIN ST, , 98662)",null,null,null,"List(, , , Individual, , , , , , , )","List(, , , Organization, BH CLINIC OF VANCOUVER, 1122334455, , , , , )","List(, , , Individual, PROVIDER SUSAN , 1112223346, PE, , , 261QR0405X, )"
ZZ,123456789,CLEARINGHOUSE,20180508:0833,222,,,234,1805080AV3648353,,,53:B:1,,,,,,F251,,VANCOUVER,19881031,D8,F,PATIENT SUSAN E,,WA,765 43RD ST,18,986640002,2,CHPWA,PI,COMMUNITY HEALTH PLAN OF WASHINGTON,null,"List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 12345 MAIN ST, , 98662)",null,null,null,"List(, , , Individual, , , , , , , )","List(, , , Organization, BH CLINIC OF VANCOUVER, 1122334455, , , , , )","List(, , , Individual, PROVIDER SUSAN , 1112223346, PE, , , 251S00000X, )"
ZZ,123456789,CLEARINGHOUSE,20180508:0833,222,,,20,1805080AV3648355,,,57:B:1,,,,,,F1020,,VANCOUVER,19881225,D8,M,SUBSCRIBER JOHN J,,WA,987 65TH PL,18,986640001,2,CHPWA,PI,COMMUNITY HEALTH PLAN OF WASHINGTON,null,"List(VANCOUVER, 720000000, EI, Organization, BH CLINIC OF VANCOUVER, 1122334455, , WA, 12345 MAIN ST, , 98662)",null,null,null,"List(, , , Individual, , , , , , , )","List(, , , Organization, BH CLINIC OF VANCOUVER, 1122334455, , , , , )","List(, , , Individual, PROVIDER JAMES , 1112223338, PE, , , 261QR0405X, )"
ZZ,OO000011111,000000005D,20180710:214339,222,,,1800,ABC11111,D8,,22:B:1,20180123,454,,,,G5621,,FAKE CITY,19650101,D8,M,ABCDEFGH IJKLMNOP B,,CA,123 ADDRESS3,18,908021112,2,ABCMMPIO,PI,ABCHYI OL POI,null,"List(FAKE CITY, 123456789, EI, Organization, YYYY HEALTHCARE ABC, 1222222220, , NY, 123 ADDRESS1, , 908021112)",null,null,null,"List(, , , Individual, ABCD STUVW , 1234567890, , , , , )","List(, , , Organization, ABCDEFG HIJKLMN HOSP, 1122334460, , , , , )","List(, , , Individual, TUVWX MNOPQR , 1234567891, PE, , , 367500000X, )"
ZZ,OO000011111,000000005D,20180710:214339,222,,,984,ABC111112,D8,,22:B:1,20180713,454,,,,K219,,FAKE CITY,19760101,D8,F,ABCDEFGH IJKLMNOP B,,CA,123 ADDRESS7,18,908021117,2,ABCMMPIO,PI,ABCHYI OL POI,null,"List(FAKE CITY, 543211234, EI, Organization, ABCDE EFGHIJ GROUP PC, 1222222223, , CA, 123 ADDRE

In [0]:
claim_lines_flat  = claims_flat.select(
    "Claim.claim_header.claim_id",
    explode(col("Claim.claim_lines")).alias("claim_lines")
)


claim_lines_flat = claim_lines_flat.selectExpr(
    "claim_id",
    "claim_lines.*"
)

display(claim_lines_flat)


claim_id,claim_line_number,dg_cd_pntr,line_chrg_amt,modifier_cds,place_of_service,prcdr_cd,prcdr_cd_type,revenue_cd,service_date,service_date_format,service_time,units,units_measurement
1805080AV3648339,1,1,20,,,H0003,HC,null,20180428,D8,472,1,UN
1805080AV3648347,1,1:2,50.1,HF,,96153,HC,null,20180426,D8,472,6,UN
1805080AV3648340,1,1:2,11.64,HF,,T1017,HC,null,20180427,D8,472,1,UN
1805080AV3648353,1,1,234,,,90853,HC,null,20180427,D8,472,120,UN
1805080AV3648355,1,1:2,20,,,H0003,HC,null,20180427,D8,472,1,UN
ABC11111,1,1,1800,QZ,,01710,HC,null,20180123-20180123,RD8,472,88,MJ
ABC11111,1,1,984,"AA,P3",,00731,HC,null,20180713-20180713,RD8,472,24,MJ
ABC11111,1,1,1353,AA,,00840,HC,null,20180713-20180713,RD8,472,62,MJ
ABC11111,1,1,1230,AA,,00902,HC,null,20180713-20180713,RD8,472,65,MJ
ABC11111,2,2,738,59,,62322,HC,null,20180713-20180713,RD8,472,1,UN


### Write table to Unity Catalog

In [0]:
# Define the catalog and schema names
catalog_name = "raven_catalog"
schema_name = "x12_edi_parser"

# Define the path where you want to save the claims_flat DataFrame as a UC table
uc_table_name = f"{catalog_name}.{schema_name}.claim_header"

# Save the claims_flat DataFrame as a Delta table in your Unity catalog
claim_header_flat.write.format("delta").mode("overwrite").saveAsTable(uc_table_name)